In [1]:
import pandas as pd
import json

In [3]:
flatten = lambda x: [j for i in x for j in i]

In [24]:
with open("../../data/pageID2CID_using_wikidata.json", 'r', encoding='utf-8') as f:
    pageID2CID_table = json.load(f)
    
CID2pageID_table = {}
for page_id, cids in pageID2CID_table.items():
    for cid in cids:
        if CID2pageID_table.get(cid):
             CID2pageID_table[cid] += [page_id]
        else:
            CID2pageID_table[cid] = [page_id]
    
with open("../../data/compound_train.json", 'r', encoding='utf-8') as f:
    train_raw = json.load(f)
    train_product = {str(entry['WikipediaID']): {'Name': entry['Name'], 'Product': entry['Attributes']['製造方法']} for entry in train_raw['entry']}
    pageID_set = set(train_product.keys())

with open("../../data/raw_manufacturings_using_wikidata.json", 'r', encoding='utf-8') as f:
    manufacturings = json.load(f)

In [8]:
wiki_df = pd.read_csv("../../data/train_split_words.csv")
wiki_df = wiki_df.append(pd.read_csv("../../data/valid_split_words.csv"))
wiki_df._id = wiki_df._id.astype(str)

In [9]:
with open("../../data/pageID2wikidataID.json", 'r') as f:
    pageid2wikidataid_table = json.load(f)

In [37]:
train_page_id = set([str(entry['WikipediaID']) for entry in train_raw['entry']])
print(len(train_page_id))

train_contained_prod = [str(entry['WikipediaID']) for entry in train_raw['entry'] if entry['Attributes']['製造方法']]
print(len(train_contained_prod))

train_notcontained_prod = [str(entry['WikipediaID']) for entry in train_raw['entry'] if not entry['Attributes']['製造方法']]
print(len(train_notcontained_prod))

598
322
276


In [34]:
# Wikipediaに製造方法がある & PubChem対応
rel1 = set([page_id for page_id in train_contained_prod if pageID2CID_table.get(page_id)])
print(len(rel1))

# Wikipediaに製造方法がある & PubChem非対応
rel2 = set([page_id for page_id in train_contained_prod if  pageID2CID_table.get(page_id) is None])
print(len(rel2))

# Wikipediaに製造方法がない & PubChemに対応
rel3 = set([page_id for page_id in train_notcontained_prod if pageID2CID_table.get(page_id)])
print(len(rel3))

# Wikipediaに製造方法がない & PubChem非対応
rel4 = set([page_id for page_id in train_notcontained_prod if pageID2CID_table.get(page_id) is None])
print(len(rel4))

253
69
200
76


In [46]:
# PubChemに製造方法が存在するWikipedia記事のID
manufacturing_page_id = set(flatten([CID2pageID_table[cid] for cid in manufacturings.keys() if CID2pageID_table.get(cid) and manufacturings.get(cid)]))
print(len(manufacturing_page_id))

# PubChemに製造方法が存在するトレーニングデータのWikipedia記事のID
train_manufacturing_page_id = (manufacturing_page_id & train_page_id)
print(len(train_manufacturing_page_id))

1633
211


In [51]:
{page_id: train_product[page_id] 
 for page_id in (rel1 - manufacturing_page_id)
}

{'1024773': {'Name': 'ペンタセン',
  'Product': ['フタルアルデヒドをシクロヘキサン-1,4-ジオンを脱水縮合させてペンタセンキノンとし、これをアルミニウムアマルガムで還元することにより、ペンタセンを得る',
   '前駆体から小分子を脱離させる手法',
   '高温でエチレン、テトラハロゲノベンゼン[5]または水[6]を、あるいは光照射により一酸化炭素を脱離させる方法',
   '下に150°Cに加熱して一酸化炭素を脱離させ、ペンタセンを得る反応',
   'ペンタセンは一般的な有機溶媒には溶けにくいが、1,2,4-トリクロロベンゼンのようなハロゲン化芳香族炭化水素系溶媒中に高温では溶けることが知られている。そこから小さな平板を形成させるために結晶化することができる']},
 '1041689': {'Name': 'トリフェニルメタン',
  'Product': ['ベンゼンと四塩化炭素からフリーデル・クラフツ反応によりクロロトリフェニルメタンを得て、これを塩化水素/エーテルで還元',
   'トリフェニル酢酸は、融点以上に加熱する']},
 '1052083': {'Name': '三ヨウ化窒素',
  'Product': ['一般的な合成法としてはヨウ素とアンモニアによるものが知られている',
   '窒化ホウ素とフッ化ヨウ素をトリクロロフルオロメタン中、−30°Cで反応させることによって']},
 '1162928': {'Name': 'ペンタゾール',
  'Product': ['アリールペンタゾールはジアゾニウム塩とアジ化物イオンの低温下での反応で合成できる。',
   'ベンゼンジアゾニウムカチオンに対しアジ化物イオンを作用させると、窒素ガスを放出してアジ化ベンゼンが生成する']},
 '1170449': {'Name': '5-ホスホメバロン酸',
  'Product': ['メバロン酸経路でメバロン酸からメバロン酸キナーゼによって合成']},
 '1193389': {'Name': '塩化ロジウム(III)',
  'Product': ['粉末ロジウムと塩化カリウムの混合物を550〜575℃で塩素ガスと反応させてヘキサクロロロジウム酸(III)カリウムを得る。この水溶液に水酸化カリウム水溶液を

In [64]:
{page_id: train_product[page_id]['Name']
 for page_id in (rel1 - manufacturing_page_id)
}

{'1024773': 'ペンタセン',
 '1041689': 'トリフェニルメタン',
 '1052083': '三ヨウ化窒素',
 '1162928': 'ペンタゾール',
 '1170449': '5-ホスホメバロン酸',
 '1193389': '塩化ロジウム(III)',
 '1237720': 'N-アセチルムラミン酸',
 '1244436': '酸化鉄(III)',
 '1268469': 'ホスホエノールピルビン酸',
 '1270151': 'ジヒドロキシアセトンリン酸',
 '1273610': 'フッ化アンモニウム',
 '1301304': '酢酸カリウム',
 '1318833': 'ホスホリボシル二リン酸',
 '1319163': 'リボース-5-リン酸',
 '1326610': 'サリチルアルコール',
 '1342797': '無水メリト酸',
 '1360982': 'キノキサリン',
 '1385415': 'アビエチン酸',
 '1429429': 'キサントホス',
 '1483298': 'アダムス触媒',
 '1490225': '1-デオキシ-D-キシルロース-5-リン酸',
 '1499971': 'ネペタラクトン',
 '1533538': 'ペンタニトロアニリン',
 '1553396': 'コバルトセン',
 '1563432': 'リゾホスファチジン酸',
 '1565134': '亜塩素酸塩',
 '1566859': 'メンタン',
 '1575659': '5-メチルテトラヒドロ葉酸',
 '1586835': 'ピリドキサールリン酸',
 '161388': '次亜塩素酸',
 '1625194': 'フッ化金(III)',
 '1654006': '酸化コバルト(III)',
 '1658717': '七フッ化ヨウ素',
 '1668122': '炭酸ルビジウム',
 '1668370': '水酸化ルビジウム',
 '1670979': 'アクリドン',
 '1680461': 'フッ化イリジウム(VI)',
 '1684434': '臭化バナジウム(III)',
 '1687568': 'ヨウ化バナジウム(III)',
 '1687634': 'フッ化バナジウム(III)',
 '16876